In [5]:
# работа с данными
import pandas as pd
import numpy as np

# графики
import matplotlib
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns

# настройка графиков
# matplotlib.rcParams['figure.figsize'] = (8, 8)
# sns.set_style('whitegrid')

# предупреждения
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

# tqdm
from tqdm.notebook import tqdm
# time
import itertools
# copy
from copy import deepcopy

# работа с файлами
import pickle
import json

In [6]:
#unlabeled = pd.read_csv("C:/Users/ranas/ML course/Competitions/Second competition/data/raw/unlabeled_data.csv")
train  = pd.read_csv("C:/Users/ranas/ML course/Competitions/Second competition/data/raw/train.csv")
test = pd.read_csv("C:/Users/ranas/ML course/Competitions/Second competition/data/raw/test.csv")

In [12]:
train.head()

,id,sentiment,review
0,c77111d1e3526ac0f7cf13af46b65e65,1,I thought the film could be a bit more complex...
1,c4806c50f6560aa65f15ed69a25940f2,0,...except for Jon Heder. This guy tanked the e...
2,82b113c68594c74ab8dbe630206ec28a,1,"Jim Carrey is good as usual, and even though t..."
3,3d9891905de05248dfc2550239bd4687,1,Just read the original story which is written ...
4,da8d5a23f01e129b534022149825caa2,1,I watched this film over a hundred times. It i...


In [10]:
train['review'][0]

"I thought the film could be a bit more complex,in a psychological sense perhaps, but the action and voice acting were top notch. The animation was heavy CG in many scenes, but very good ones at that. This is one of the Batman Returns/Forever type films, which include romances and the conflicts of Wayne and motives for dating. 007 fans would love this, and so would the females, great theme song! Wayne was portrayed very well in this film, and the Penquin was back to his true form, no mutant genes in him this time! I liked the fact Robin wasn't used too much, Tim Drake was just a good computer nerd, somewhat of an Indigo child or mind of the future.<br /><br />The supporting cast was made up of some soap opera stars, decent talents and the characters were drawn to look like the voice actors too. Kelly Ripa was hilarious in this film.<br /><br />I rate this below Phantasm, Return of the Joker, and Batman vs. Dracula, but liked the smarter script better than I enjoyed Subzero. 7/10"

### Vader

Valence Aware Dictionary for sEntiment Reasoning

his lexical dictionary does not only contain words, but also phrases (such as “bad ass” and “the bomb”), emoticons (such as “:-)”) and sentiment-laden acronyms (such as “ROFL” and “WTF”). All the lexical features were rated for the polarity and intensity on a scale from “-4: Extremely Negative” to “+4 Extremely Positive” by 10 independent human raters. The average score is then used as the sentiment indicator for each lexical feature in the dictionary. For example, in Vader, the word “okay” has a positive rating of 0.9, “good” is 1.9 and “great” is 3.1, whereas “horrible” is -2.5, the frowning emoticon “:(“ is -2.2, and “sucks” is -1.5. Vader’s lexicon dictionary contains around 7,500 sentiment features in total and any word not listed in the dictionary will be scored as “0: Neutral”.

In [7]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ranas\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [8]:
sia.polarity_scores('I am so happy!')

{'neg': 0.0, 'neu': 0.318, 'pos': 0.682, 'compound': 0.6468}

In [11]:
sia.polarity_scores(train['review'][0])

{'neg': 0.039, 'neu': 0.663, 'pos': 0.298, 'compound': 0.9963}

In [13]:
sia.polarity_scores(train['review'][1])

{'neg': 0.188, 'neu': 0.689, 'pos': 0.123, 'compound': -0.8989}

In [14]:
sia.polarity_scores(train['review'][2])

{'neg': 0.04, 'neu': 0.799, 'pos': 0.161, 'compound': 0.7904}

In [15]:
sia.polarity_scores(train['review'][3])

{'neg': 0.021, 'neu': 0.728, 'pos': 0.251, 'compound': 0.9437}

In [16]:
sia.polarity_scores(train['review'][4])

{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.9694}

А оно неплохо справляется!

In [17]:
res = {}
for i, row in tqdm(train.iterrows(), total=len(train)):
    text = row['review']
    myid = row['id']
    res[myid] = sia.polarity_scores(text)

  0%|          | 0/12500 [00:00<?, ?it/s]

In [19]:
vaders = pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={'index': 'id'})
vaders = vaders.merge(train, how='left')

In [20]:
vaders.head()

,id,neg,neu,pos,compound,sentiment,review
0,c77111d1e3526ac0f7cf13af46b65e65,0.039,0.663,0.298,0.9963,1,I thought the film could be a bit more complex...
1,c4806c50f6560aa65f15ed69a25940f2,0.188,0.689,0.123,-0.8989,0,...except for Jon Heder. This guy tanked the e...
2,82b113c68594c74ab8dbe630206ec28a,0.040,0.799,0.161,0.7904,1,"Jim Carrey is good as usual, and even though t..."
3,3d9891905de05248dfc2550239bd4687,0.021,0.728,0.251,0.9437,1,Just read the original story which is written ...
4,da8d5a23f01e129b534022149825caa2,0.000,0.508,0.492,0.9694,1,I watched this film over a hundred times. It i...


In [21]:
vaders['predict'] = np.where(vaders['compound']>0,1,0)

In [25]:
from sklearn.metrics import accuracy_score

train_metric = accuracy_score(
    y_true=vaders['sentiment'],
    y_pred=vaders['predict']
)

print(f"Accuracy score: {round(train_metric, 4)}")

Accuracy score: 0.6956


### Roberta pretrained model

In [26]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [27]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

C:\Users\ranas\anaconda3\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ranas\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [28]:
encoded_text = tokenizer(train['review'][0], return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
}
print(scores_dict)

{'roberta_neg': 0.0050721676, 'roberta_neu': 0.047950763, 'roberta_pos': 0.9469772}


In [33]:
train['review_length'] = train['review'].apply(lambda x: len(x))

In [34]:
train.head()

,id,sentiment,review,review_length
0,c77111d1e3526ac0f7cf13af46b65e65,1,I thought the film could be a bit more complex...,992
1,c4806c50f6560aa65f15ed69a25940f2,0,...except for Jon Heder. This guy tanked the e...,842
2,82b113c68594c74ab8dbe630206ec28a,1,"Jim Carrey is good as usual, and even though t...",494
3,3d9891905de05248dfc2550239bd4687,1,Just read the original story which is written ...,370
4,da8d5a23f01e129b534022149825caa2,1,I watched this film over a hundred times. It i...,177


In [37]:
train['review_length'].describe()

count    12500.000000
mean      1316.745120
std        998.130579
min         52.000000
25%        701.000000
50%        970.000000
75%       1593.000000
max       9432.000000
Name: review_length, dtype: float64

In [38]:
train[train['id']=='2da40a4b5698f815c98198e05bd7652b']

,id,sentiment,review,review_length
26,2da40a4b5698f815c98198e05bd7652b,0,After just finishing the book the same day I w...,2373


In [39]:
import sys
sys.path.insert(1, 'C:/Users/ranas/ML course/Competitions/Second competition/functions')

In [66]:
from data_preprocess import light_text_cleaning
train = light_text_cleaning(train, 'review')

  0%|          | 0/12500 [00:00<?, ?it/s]

In [62]:
train.head()

,id,sentiment,review,review_length,tidy_review,tidy_review_length
0,c77111d1e3526ac0f7cf13af46b65e65,1,I thought the film could be a bit more complex...,992,I thought the film could be a bit more complex...,960
1,c4806c50f6560aa65f15ed69a25940f2,0,...except for Jon Heder. This guy tanked the e...,842,...except for Jon Heder. This guy tanked the e...,816
2,82b113c68594c74ab8dbe630206ec28a,1,"Jim Carrey is good as usual, and even though t...",494,"Jim Carrey is good as usual, and even though t...",470
3,3d9891905de05248dfc2550239bd4687,1,Just read the original story which is written ...,370,Just read the original story which is written ...,368
4,da8d5a23f01e129b534022149825caa2,1,I watched this film over a hundred times. It i...,177,I watched this film over a hundred times. It i...,177


In [42]:
train['tidy_review_length'] = train['tidy_review'].apply(lambda x: len(x))
train['tidy_review_length'].describe()

count    12500.000000
mean      1285.745840
std        974.362788
min         52.000000
25%        688.000000
50%        948.000000
75%       1557.250000
max       9331.000000
Name: tidy_review_length, dtype: float64

In [43]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, max_length = 511, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [44]:
res = {}
for i, row in tqdm(train.iterrows(), total=len(train)):
    try:
        text = row['review']
        myid = row['id']
        roberta_result = polarity_scores_roberta(text)
        res[myid] = roberta_result
    except RuntimeError:
        print(f'Broke for id {myid}')
        res[myid] = {
        'roberta_neg' : 0.5,
        'roberta_neu' : 0.5,
        'roberta_pos' : 0.5
        }
        

  0%|          | 0/12500 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


KeyboardInterrupt: 

In [47]:
results_df = pd.DataFrame(res).T
results_df = results_df.reset_index().rename(columns={'index': 'id'})
results_df = results_df.merge(train, how='inner')

In [48]:
results_df

,id,roberta_neg,roberta_neu,roberta_pos,sentiment,review,review_length,tidy_review,tidy_review_length
0,c77111d1e3526ac0f7cf13af46b65e65,0.005072,0.047951,0.946977,1,I thought the film could be a bit more complex...,992,I thought the film could be a bit more complex...,960
1,c4806c50f6560aa65f15ed69a25940f2,0.476714,0.350946,0.172340,0,...except for Jon Heder. This guy tanked the e...,842,...except for Jon Heder. This guy tanked the e...,816
2,82b113c68594c74ab8dbe630206ec28a,0.046676,0.100293,0.853030,1,"Jim Carrey is good as usual, and even though t...",494,"Jim Carrey is good as usual, and even though t...",470
3,3d9891905de05248dfc2550239bd4687,0.003896,0.050801,0.945302,1,Just read the original story which is written ...,370,Just read the original story which is written ...,368
4,da8d5a23f01e129b534022149825caa2,0.001529,0.008747,0.989724,1,I watched this film over a hundred times. It i...,177,I watched this film over a hundred times. It i...,177
...,...,...,...,...,...,...,...,...,...
4995,a84e26db8a4410059b521dff427ec7c1,0.961930,0.034520,0.003550,0,This movie was bad from the start. The only pu...,760,This movie was bad from the start. The only pu...,760
4996,9c1a0d87818cc8a77d3c9acb8ff33452,0.751652,0.215149,0.033199,0,After having seen the movie the first question...,601,After having seen the movie the first question...,598
4997,a272bcf0561628bd50590b52e0bc657a,0.840266,0.138615,0.021119,0,How do comments like the one that was the head...,1352,How do comments like the one that was the head...,1352
4998,de606367278a2a23578d188a55f733de,0.025093,0.183860,0.791047,1,A masterful treatment of James Caine's \The Po...,862,A masterful treatment of James Caine's \The Po...,846


In [49]:
from sklearn.metrics import roc_auc_score
roc_auc_score(results_df['sentiment'], results_df['roberta_pos'])

0.9285581942327769

Так долго ждать, а результат не очень-то уже и хороший

Посмотрим на чистоту данных ещё раз

In [68]:
train['tidy_review'][2]

'Jim Carrey is good as usual, and even though there are quite a few \\Jim Carrey moments\\", it\'s definitely not a \\"Jim Carrey movie\\".It\'s targeted mostly at children, and I managed to enjoy it as such movie. It was promoted in Israel as another Jim Carrey movie, so those who expected a weird over the top comedy were disappointed.The movie has nice moments and works well as a movie for kids. I can\'t say I LOVED this movie, but then again I\'m not its target audience!"'

Сделаем случайную выборку с довольно маленькими отзывами

In [52]:
train.head()

,id,sentiment,review,review_length,tidy_review,tidy_review_length
0,c77111d1e3526ac0f7cf13af46b65e65,1,I thought the film could be a bit more complex...,992,I thought the film could be a bit more complex...,960
1,c4806c50f6560aa65f15ed69a25940f2,0,...except for Jon Heder. This guy tanked the e...,842,...except for Jon Heder. This guy tanked the e...,816
2,82b113c68594c74ab8dbe630206ec28a,1,"Jim Carrey is good as usual, and even though t...",494,"Jim Carrey is good as usual, and even though t...",470
3,3d9891905de05248dfc2550239bd4687,1,Just read the original story which is written ...,370,Just read the original story which is written ...,368
4,da8d5a23f01e129b534022149825caa2,1,I watched this film over a hundred times. It i...,177,I watched this film over a hundred times. It i...,177


In [ ]:
train_sample = train